# Setup

In [1]:
import hydroDL.data.dbVeg
from hydroDL.data import dbVeg
import importlib
import numpy as np
import json
import os
from hydroDL import utils
from hydroDL.post import mapplot, axplot, figplot
import matplotlib.pyplot as plt
from hydroDL.model import rnn, crit, trainBasin
import math
import torch
from torch import nn
from hydroDL.data import DataModel
from hydroDL.master import basinFull, slurm, dataTs2Range
import torch.optim as optim
from hydroDL import kPath
import torch.optim.lr_scheduler as lr_scheduler
import dill
from tqdm import tqdm

loading package hydroDL


In [2]:
rho = 45 # init rho
dataName = 'singleDaily' # init dataName
importlib.reload(hydroDL.data.dbVeg) # reimport library
df = dbVeg.DataFrameVeg(dataName) # create DataFrameVeg class 
dm = DataModel(X=df.x, XC=df.xc, Y=df.y) # (?) create DataModel class (contains many confusing functions) 
siteIdLst = df.siteIdLst # get site list
dm.trans(mtdDefault='minmax') # (?) some sort of data normalization
dataTup = dm.getData() # get x, xc, y, and yc
dataEnd, (iInd, jInd) = dataTs2Range(dataTup, rho, returnInd=True) # get data into form (# LFMC, 91 day window, varX) 
x, xc, y, yc = dataEnd # data from dataTs2Range

In [3]:
# get indices of variables of interest
varS = ['VV', 'VH', 'vh_vv']
varL = ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'ndvi', 'ndwi', 'nirv']
varM = ['Fpar', 'Lai']
iS = [df.varX.index(var) for var in varS]
iL = [df.varX.index(var) for var in varL]
iM = [df.varX.index(var) for var in varM]

In [4]:
# for each satellite, for each LFMC data point
# create a list of days in the 91 day window that have data

# nMat -- (# LFMC, # satellites)
# nMat contains # of days each satellite has data for
pSLst, pLLst, pMLst = list(), list(), list()
ns = yc.shape[0]
nMat = np.zeros([yc.shape[0], 3])
for k in range(nMat.shape[0]):
    tempS = x[:, k, iS] # x (rho, LFMC, varX) 
    pS = np.where(~np.isnan(tempS).any(axis=1))[0]
    tempL = x[:, k, iL] # x (rho, LFMC, varX) 
    pL = np.where(~np.isnan(tempL).any(axis=1))[0]
    tempM = x[:, k, iM] # x (rho, LFMC, varX) 
    pM = np.where(~np.isnan(tempM).any(axis=1))[0]
    pSLst.append(pS)
    pLLst.append(pL)
    pMLst.append(pM)
    nMat[k, :] = [len(pS), len(pL), len(pM)]

In [5]:
# only keep if data if there is at least 1 day of data for 
# each satellite
indKeep = np.where((nMat > 0).all(axis=1))[0]
x = x[:, indKeep, :]
xc = xc[indKeep, :]
yc = yc[indKeep, :]
nMat = nMat[indKeep, :]
pSLst = [pSLst[k] for k in indKeep]
pLLst = [pLLst[k] for k in indKeep]
pMLst = [pMLst[k] for k in indKeep]
jInd = jInd[indKeep]

# update from just list of sites to sites per datapoint
siteIdLst = [siteIdLst[k] for k in jInd] 

In [6]:
jSite, count = np.unique(jInd, return_counts=True) # sites, # of times site appears
countAry = np.array([[x, y] for y, x in sorted(zip(count, jSite))]) # rearrange
nRm = sum(countAry[:, 1] < 5) # # of sites that show up less than 5 times
indSiteAll = countAry[nRm:, 0].astype(int) # remove sites that show up less than 5 times
dictSubset = dict()

# create 5 folds, each with train and test data
for k in range(5):
    siteTest = indSiteAll[k::5] 
    siteTrain = np.setdiff1d(indSiteAll, siteTest)
    indTest = np.where(np.isin(jInd, siteTest))[0]
    indTrain = np.where(np.isin(jInd, siteTrain))[0]
    dictSubset['testSite_k{}5'.format(k)] = siteTest.tolist()
    dictSubset['trainSite_k{}5'.format(k)] = siteTrain.tolist()
    dictSubset['testInd_k{}5'.format(k)] = indTest.tolist()
    dictSubset['trainInd_k{}5'.format(k)] = indTrain.tolist()

array([ 40,  45, 215, 300, 305, 322,  11, 120, 164, 181, 189, 199, 204,
       244, 291, 293,  39,  44,  46, 118, 123, 147, 162, 167, 177, 179,
       198, 207, 287, 294, 297, 298, 315,  48, 192, 292, 295, 299,  33,
        49,  50, 176, 191, 200, 208, 290, 313, 193, 296, 308, 320, 106,
       144, 180, 182, 174, 175, 319,  47, 157, 159, 161, 201, 158, 188,
        30, 316,  38, 140, 165, 186, 225, 286, 307, 318,  36,  41,  42,
        43, 209, 210, 211, 213, 282, 314, 317,  92, 122, 125, 127,  21,
        93, 270, 252, 255, 271, 133, 325, 254, 257,  16,  26,  96, 119,
       124, 146, 149,  95,  97, 121, 148,  10,  32,  90,  98, 129, 185,
       217,  20,  81, 166, 272,  86,  91, 131, 274,   9,  19,  35,  87,
       155,  34,  37,  85, 156, 276, 277,  22,  82, 259, 273, 275, 281,
        58, 260,  13,  23,  84,  88, 151, 216, 236, 237, 261,   5, 170,
       249, 250, 263,  60,   6,  70, 171, 251, 288, 301,  27,  28,  62,
       126, 168, 226,  72, 239,  56,  69,  71, 221, 224, 267, 26

In [66]:
tInd = iInd
siteInd = jInd
trainInd = dictSubset['trainInd_k05']
testInd = dictSubset['testInd_k05']

# Random subset

In [67]:
def randomSubset(opt='train', batch=1000):
    # random sample within window
    varS = ['VV', 'VH', 'vh_vv']
    varL = ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'ndvi', 'ndwi', 'nirv']
    varM = ['Fpar', 'Lai'] 
    iS = [df.varX.index(var) for var in varS]
    iL = [df.varX.index(var) for var in varL]
    iM = [df.varX.index(var) for var in varM]
    
    if opt == 'train':
        indSel = np.random.permutation(trainInd)[0:batch]
    else:
        indSel = testInd
    ns = len(indSel)

    # Step 1: Create a ns by bX matrix of vals 0 to nMat[indSel, X]
    # random.randint(low, high=None, size=None)
    rS = np.random.randint(0, nMat[indSel, 0], [bS, ns]).T
    rL = np.random.randint(0, nMat[indSel, 1], [bL, ns]).T
    rM = np.random.randint(0, nMat[indSel, 2], [bM, ns]).T
    
    pS = np.stack([pSLst[indSel[k]][rS[k, :]] for k in range(ns)], axis=0)
    pL = np.stack([pLLst[indSel[k]][rL[k, :]] for k in range(ns)], axis=0)
    pM = np.stack([pMLst[indSel[k]][rM[k, :]] for k in range(ns)], axis=0)
    
    matS1 = x[:, indSel, :][:, :, iS]
    matL1 = x[:, indSel, :][:, :, iL]
    matM1 = x[:, indSel, :][:, :, iM]
    
    xS = np.stack([matS1[pS[k, :], k, :] for k in range(ns)], axis=0)
    xL = np.stack([matL1[pL[k, :], k, :] for k in range(ns)], axis=0)
    xM = np.stack([matM1[pM[k, :], k, :] for k in range(ns)], axis=0)
    
    pS = (pS - rho) / rho
    pL = (pL - rho) / rho
    pM = (pM - rho) / rho
    
    return (
        torch.tensor(xS, dtype=torch.float32),
        torch.tensor(xL, dtype=torch.float32),
        torch.tensor(xM, dtype=torch.float32),
        torch.tensor(pS, dtype=torch.float32),
        torch.tensor(pL, dtype=torch.float32),
        torch.tensor(pM, dtype=torch.float32),
        torch.tensor(xc[indSel, :], dtype=torch.float32),
        torch.tensor(yc[indSel, 0], dtype=torch.float32),
    )

### Exploration

In [82]:
opt = "train"
batch = 1000

In [83]:
if opt == 'train':
    indSel = np.random.permutation(trainInd)[0:batch]
else:
    indSel = testInd
ns = len(indSel)

In [92]:
ns

1000

In [100]:
# for each lfmc in batch, get 8 random vals (w/ rep) from [0, # days for lfmc]
rS = np.random.randint(0, nMat[indSel, 0], [bS, ns]).T

In [101]:
nMat[indSel, 0].shape

(1000,)

In [102]:
rS.shape

(1000, 8)

In [123]:
# for each lfmc in batch, choose 8 days based on vals in rS
pS = np.stack([pSLst[indSel[k]][rS[k, :]] for k in range(ns)], axis=0)

In [124]:
pS.shape

(1000, 8)

In [135]:
# pSLst[ index an lfmc point in the dataset ][ index bS random days ] 
# pSLst -- for each LFMC data point, a list of days in the 91 day window that have data

In [125]:
matS1 = x[:, indSel, :][:, :, iS]

In [126]:
x.shape # (rho, #lfmc, #var)

(91, 9970, 20)

In [128]:
# select examples in batch
x[:, indSel, :].shape  # (rho, batch size, #var)

(91, 1000, 20)

In [129]:
# select variables of interest
x[:, indSel, :][:, :, iS].shape # (rho, batch size, #var desired)

(91, 1000, 3)

In [131]:
xS = np.stack([matS1[pS[k, :], k, :] for k in range(ns)], axis=0)

In [132]:
# for example k, get the 8 days sampled
pS[k, :].shape

(8,)

In [134]:
# for example k, index matS1 for [8 days, example k, all vars of interst]
matS1[pS[k, :], k, :].shape

(8, 3)

In [136]:
xS.shape # (# examples, 8 days, 3 variables of intest)

(1000, 8, 3)

In [142]:
pS = (pS - rho) / rho
pS.shape
# normalize chose days

(1000, 8)

In [143]:
# xS -- (1000, bS, 3)
# xL -- (1000, bL, 8)
# xM -- (1000, bM, 2)

# pS -- (1000, bS)
# pL -- (1000, bL)
# pM -- (1000, bM)

# xc -- (1000, 15)
# yc -- (1000, 1)

# Model

In [68]:
class InputFeature(nn.Module):
    def __init__(self, nTup, nxc, nh):
        # nTup -- # of inputs for each satellite
        # nxc -- # of const vars
        # nh -- # hidden layers
        super().__init__()
        self.nh = nh
        self.lnXc = nn.Sequential(nn.Linear(nxc, nh), nn.ReLU(), nn.Linear(nh, nh))
        self.lnLst = nn.ModuleList()
        for n in nTup:
            self.lnLst.append(
                nn.Sequential(nn.Linear(n, nh), nn.ReLU(), nn.Linear(nh, nh))
            )

    def getPos(self, pos):
        nh = self.nh
        P = torch.zeros([pos.shape[0], pos.shape[1], nh], dtype=torch.float32)
        for i in range(int(nh / 2)):
            P[:, :, 2 * i] = torch.sin(pos / (i + 1) * torch.pi)
            P[:, :, 2 * i + 1] = torch.cos(pos / (i + 1) * torch.pi)
        return P

    def forward(self, xTup, pTup, xc):
        outLst = list()
        for k in range(len(xTup)):
            x = self.lnLst[k](xTup[k]) + self.getPos(pTup[k])
            outLst.append(x)
        outC = self.lnXc(xc)
        # outLst[0].shape -> torch.Size([1000, 8, 32])
        # outLst[1].shape -> torch.Size([1000, 6, 32])
        # outLst[2].shape -> torch.Size([1000, 10, 32])
        # outC[:, None, :].shape -> torch.Size([1000, 1, 32])
        # out.shape -> torch.Size([1000, 25, 32])
        out = torch.cat(outLst + [outC[:, None, :]], dim=1)
        return out


class AttentionLayer(nn.Module):
    def __init__(self, nx, nh):
        super().__init__()
        self.nh = nh
        self.W_k = nn.Linear(nx, nh, bias=False)
        self.W_q = nn.Linear(nx, nh, bias=False)
        self.W_v = nn.Linear(nx, nh, bias=False)
        self.W_o = nn.Linear(nh, nh, bias=False)

    def forward(self, x):
        q, k, v = self.W_q(x), self.W_k(x), self.W_v(x)
        d = q.shape[1]
        score = torch.bmm(q.transpose(1, 2), k) / math.sqrt(d)
        attention = torch.softmax(score, dim=-1)
        out = torch.bmm(attention, v.transpose(1, 2))
        out = self.W_o(out.transpose(1, 2))
        return out


class PositionWiseFFN(nn.Module):
    def __init__(self, nh, ny):
        super().__init__()
        self.dense1 = nn.Linear(nh, nh)
        self.relu = nn.ReLU()
        self.dense2 = nn.Linear(nh, ny)

    def forward(self, X):
        return self.dense2(self.relu(self.dense1(X)))


class AddNorm(nn.Module):
    def __init__(self, norm_shape, dropout):
        super().__init__()
        self.dropout = nn.Dropout(dropout)
        self.ln = nn.LayerNorm(norm_shape)

    def forward(self, X, Y):
        return self.ln(self.dropout(Y) + X)


class FinalModel(nn.Module):
    def __init__(self, nTup, nxc, nh):
        # nTup -- # of inputs for each satellite
        # nxc -- # of const vars
        # nh -- # hidden layers
        super().__init__()
        self.nTup = nTup
        self.nxc = nxc
        self.encoder = InputFeature(nTup, nxc, nh)
        self.atten = AttentionLayer(nh, nh)
        self.addnorm1 = AddNorm(nh, 0.1)
        self.addnorm2 = AddNorm(nh, 0.1)
        self.ffn1 = PositionWiseFFN(nh, nh)
        self.ffn2 = PositionWiseFFN(nh, 1)
        for p in self.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)

    def forward(self, x, pos, xcT, lTup):
        xIn = self.encoder(x, pos, xcT)
        out = self.atten(xIn)
        out = self.addnorm1(xIn, out)
        out = self.ffn1(out)
        out = self.addnorm2(xIn, out)
        out = self.ffn2(out)
        out = out.squeeze(-1)
        k = 0
        temp = 0
        for i in lTup:
            temp = temp + out[:, k : i + k].mean(-1)
            k = k + i
        temp = temp + out[:, k:].mean(-1)
        return temp

# Training

In [69]:
bS = 8
bL = 6
bM = 10
nh = 32

In [71]:
xS, xL, xM, pS, pL, pM, xcT, yT = randomSubset()
nTup = (xS.shape[-1], xL.shape[-1], xM.shape[-1])
lTup = (xS.shape[1], xL.shape[1], xM.shape[1])
nxc = xc.shape[-1]

In [73]:
model = FinalModel(nTup, nxc, nh)
yP = model((xS, xL, xM), (pS, pL, pM), xcT, lTup)

In [74]:
loss_fn = nn.L1Loss(reduction='mean')
learning_rate = 1e-2
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = lr_scheduler.LinearLR(
    optimizer, start_factor=1.0, end_factor=0.01, total_iters=800
)

In [ ]:
model.train()
nEp = 500
nIterEp = 20
for ep in range(nEp):
    lossEp = 0
    for i in range(nIterEp):
        t0 = time.time()
        xS, xL, xM, pS, pL, pM, xcT, yT = randomSubset()
        t1 = time.time()
        model.zero_grad()
        yP = model((xS, xL, xM), (pS, pL, pM), xcT, lTup)
        loss = loss_fn(yP, yT)
        loss.backward()
        t2 = time.time()
        lossEp = lossEp + loss.item()
        optimizer.step()
    optimizer.zero_grad()
    xS, xL, xM, pS, pL, pM, xcT, yT = randomSubset('test')
    yP = model((xS, xL, xM), (pS, pL, pM), xcT, lTup)
    loss = loss_fn(yP, yT)
    corr = np.corrcoef(yP.detach().numpy(), yT.detach().numpy())[0, 1]
    if ep > 200:
        scheduler.step()
    print(
        '{} {:.3f} {:.3f} {:.3f} time {:.2f} {:.2f}'.format(
            ep, lossEp / nIterEp, loss.item(), corr, t1 - t0, t2 - t1
        )
    )